# Übung 7

## Schnittstellen/API

Mit der Pageviews API von Wikipedia können verschiedene Abrufstatistiken abgerufen werden.
Mach dich zuerst mit der [Pageviews-Dokumentation](https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews) vertraut.
Bearbeite anschließend die Aufgabe zum Wikipedia Jahresrückblick.


Welche Wikipedia-Seiten wurden an deinem Geburtstag am häufigsten in Deutschland besucht?
Speichere die Top 10 in einer Liste.

In [ ]:
# Dein Code


Ermittle nun für jede Seite der Top 10 wie häufig diese zwischen dem 1. Januar und dem 30. November 2022 aufgerufen wurde.

In [ ]:
# Dein Code


Speichere die Ergebnisse in einem pandas DataFrame und stelle die Aufrufzahlen der Top 10 in dem Zeitraum grafisch dar.

In [ ]:
# Dein Code


## REST-Abfragen auf normalen Webseiten

Stelle eine get-request an deine Liebelingswebseite und gib den Status Code der Antwort aus.

In [ ]:
# Dein Code


Gib nun den Inhalt der Antwort aus.

In [ ]:
# Dein Code


Versuche den eben ausgeben Inhalt im JSON-Format auszugeben.

In [ ]:
# Dein Code
